# Company Rating Parsers

In [679]:
!pip install pygsheets
!pip install https://github.com/nithinmurali/pygsheets/archive/staging.zip

  Using cached https://github.com/nithinmurali/pygsheets/archive/staging.zip


In [681]:
import pygsheets
from datetime import timedelta
import numpy as np

from time import sleep
import requests

import pandas as pd
from bs4 import BeautifulSoup


## 1/ Hourly Basis Parser

In [683]:
def soup2list(src, list_, attr=None):
    if attr:
        for val in src:
            list_.append(val[attr])
    else:
        for val in src:
            list_.append(val.get_text())

company = 'google.com' # COMPANY NAME OF CHOICE
hours = 0

while hours < 3: # number of hours to parse
    
    overall_rating = []
    nbr_by_stars = []
    nbr_by_stars_int = []
    nbr_by_stars_fin = []
    total_NBR = 0
    
    # Parsing Overall Rating
    rating = requests.get(fr"https://www.trustpilot.com/review/{company}?page=1")
    soup_NR = BeautifulSoup(rating.content)
    soup2list(soup_NR.find_all('span', {'class', 'typography_heading-m__T_L_X typography_appearance-default__AAY17'}), overall_rating)

    overall_rating_f = float(overall_rating[0])

    # Parsing for Number of Stars
    reviews_by_number_of_stars = requests.get(fr"https://www.trustpilot.com/review/{company}?page=1")
    soup_NRS = BeautifulSoup(reviews_by_number_of_stars.content)
    soup2list(soup_NRS.find_all('label', {'class', 'styles_row__wvn4i'}), nbr_by_stars, attr='title')

    for i in nbr_by_stars:
        i = i.split()
        nbr_by_stars_int.append(i)

    for j in nbr_by_stars_int:
        nbr_by_stars_fin.append(int(j[0].replace(",", "")))

    # Total Number of Reviews
    for k in nbr_by_stars_fin:
        total_NBR += k
    
    # For demonstration purposes
    
    print(f'Overall Rating: {overall_rating_f}')
    print(f'Total Number of Reviews: {total_NBR}')
    print(f'Reviews by Stars 5, 4, 3, 2, 1, accordingly: {nbr_by_stars_fin}')

    # Writing Down the Data 
    gc = pygsheets.authorize(service_file='/Users/zvezdaetogodoma/Desktop/company-ratings-project-6135c223419a.json')

    # Create empty dataframe
    df = pd.DataFrame()

    # initialize data of lists. 
    data = {'Category': ['Overall Rating', 'Total Number of Reviews', '5 Stars Reviews', '4 Stars Reviews', '3 Stars Reviews', '2 Stars Reviews', '1 Star Reviews'], 
            'Value': [f'{overall_rating_f}', f'{total_NBR}', f'{nbr_by_stars_fin[0]}', f'{nbr_by_stars_fin[1]}', f'{nbr_by_stars_fin[2]}', f'{nbr_by_stars_fin[3]}', f'{nbr_by_stars_fin[4]}']
           } 

    # Create DataFrame 
    df = pd.DataFrame(data)

    #open the google spreadsheet (where 'Company_ratings' is the name of my sheet)
    sh = gc.open('Company_ratings')

    #select the first sheet
    wks = sh[0]

    #update the first sheet with df, starting at cell B2. 
    wks.set_dataframe(df,(1,1))
    
    hours += 1
    overall_rating.clear()
    nbr_by_stars.clear()
    nbr_by_stars_int.clear()
    nbr_by_stars_fin.clear()

    total_NBR = 0
    
    sleep(3600) # one hour gap in seconds
    

Overall Rating: 3.6
Total Number of Reviews: 7410
Reviews by Stars 5, 4, 3, 2, 1, accordingly: [4315, 533, 234, 253, 2075]
Overall Rating: 3.6
Total Number of Reviews: 7410
Reviews by Stars 5, 4, 3, 2, 1, accordingly: [4315, 533, 234, 253, 2075]
Overall Rating: 3.6
Total Number of Reviews: 7410
Reviews by Stars 5, 4, 3, 2, 1, accordingly: [4315, 533, 234, 253, 2075]


## 2/ Parser for Month Analysis & Graph

In [684]:
def soup2list(src, list_, attr=None):
    if attr:
        for val in src:
            list_.append(val[attr])
    else:
        for val in src:
            list_.append(val.get_text())

company = 'google.com'
ratings = []
dates = []
last_page = []
review_dates = []
review_dates_int = []

# pagef = requests.get(fr"https://www.trustpilot.com/review/{company}?page=1")
pagef = requests.get(fr"https://www.trustpilot.com/review/{company}?date=last30days&languages=all&sort=recency")

soup_fp = BeautifulSoup(pagef.content)
soup2list(soup_fp.find_all('span', {'class', 'typography_heading-xxs__QKBS8 typography_appearance-inherit__D7XqR typography_disableResponsiveSizing__OuNP7'}), last_page)

from_page = 1
to_page = int(last_page[9])
# print(to_page)


for i in range(from_page, to_page+1):

    result = requests.get(fr"https://www.trustpilot.com/review/{company}?date=last30days&languages=all&page={i}&sort=recency")
    soup = BeautifulSoup(result.content)

    soup2list(soup.find_all('div', {'class','styles_reviewHeader__iU9Px'}), dates)
    soup2list(soup.find_all('div', {'class','styles_reviewHeader__iU9Px'}), ratings, attr='data-service-review-rating')

    sleep(2)

for review_date in dates:
    review_date = review_date.replace("Updated ", "")
    review_date = review_date.replace("Redirected", "")

    if "hours ago" in review_date.lower() or "hour ago" in review_date.lower() or "minutes ago" in review_date.lower() or "minute ago" in review_date.lower():
        review_date = dt.datetime.now().date()
    elif "a day ago" in review_date.lower():
        review_date = dt.datetime.now().date() - dt.timedelta(days=1)
    elif "days ago" in review_date.lower():
        review_date = dt.datetime.now().date() - dt.timedelta(days=int(review_date[0]))
    else:
        review_date = dt.datetime.strptime(review_date, "%b %d, %Y").date()

    review_dates.append(review_date)

ratings_int = [int(x) for x in ratings]
    

review_data = pd.DataFrame(
{
    'date':review_dates,
    'Rating': ratings_int
})

## Every Day Data Processing For Graph

In [685]:
# review_data.sort_values(by='date', inplace = True)

out = review_data.groupby('date', as_index=False)['Rating'].mean()
out.Rating = out.Rating.round(1)

out['Overall Reviews per day'] = out['date'].map(review_data['date'].value_counts())

print(out)

out['Rating'] = out['Rating'].astype(int)
# print(out.dtypes)

# Writing Down the Data 
gc = pygsheets.authorize(service_file='/Users/zvezdaetogodoma/Desktop/company-ratings-project-6135c223419a.json')

#open the google spreadsheet (where 'Company_ratings' is the name of my sheet)
sh = gc.open('Company_ratings')

#select the third sheet
wks = sh[2]

#update the first sheet with df, starting at cell B2. 
wks.set_dataframe(out,(1,1))
    

          date  Rating  Overall Reviews per day
0   2023-10-14     3.0                        2
1   2023-10-15     1.7                        3
2   2023-10-16     3.4                        5
3   2023-10-17     3.0                        4
4   2023-10-18     4.0                        4
5   2023-10-19     1.0                        2
6   2023-10-20     1.0                        2
7   2023-10-21     3.0                        2
8   2023-10-22     3.0                        3
9   2023-10-23     4.8                        4
10  2023-10-24     4.7                        3
11  2023-10-25     2.3                        3
12  2023-10-26     5.0                        2
13  2023-10-27     4.2                        5
14  2023-10-28     4.7                        3
15  2023-10-29     5.0                        5
16  2023-10-30     3.5                        8
17  2023-10-31     2.0                        4
18  2023-11-01     1.2                        4
19  2023-11-02     1.6                  

## Weekly Basis Data Processing

In [686]:
review_data['date'] = pd.to_datetime(review_data['date'])
review_data['date'].max()

start_dt = review_data['date'].min()
end_dt = review_data['date'].max()

# difference between current and previous date
delta = timedelta(days=1)

# store the dates between two dates in a list
month_dates = []

while start_dt <= end_dt:
    # add current date to list by converting  it to iso format
    month_dates.append(start_dt.isoformat())
    # increment start date by timedelta
    start_dt += delta

week1 = month_dates[0:7]
week2 = month_dates[7:14]
week3 = month_dates[14:21]
week4 = month_dates[21:28]
week5 = month_dates[28:]

conditions = [
    (review_data['date'] <= week1[-1]),
    (review_data['date'] >= week2[0]) & (review_data['date'] <= week2[-1]),
    (review_data['date'] >= week3[0]) & (review_data['date'] <= week3[-1]),
    (review_data['date'] >= week4[0]) & (review_data['date'] <= week4[-1]),
    (review_data['date'] >= week5[0]) & (review_data['date'] <= week5[-1])
    ]

# create a list of the values we want to assign for each condition
values = [1, 2, 3, 4, 5]

# create a new column and use np.select to assign values to it using our lists as arguments
review_data['week_number'] = np.select(conditions, values)

# display updated DataFrame

review_data.sort_values(by='date', inplace = True) 

review_data

,date,Rating,week_number
126,2023-10-14,5,1
125,2023-10-14,1,1
124,2023-10-15,3,1
123,2023-10-15,1,1
122,2023-10-15,1,1
...,...,...,...
14,2023-11-11,2,5
2,2023-11-11,5,5
11,2023-11-11,1,5
13,2023-11-12,5,5


## Percentage Calculations

In [687]:
out2 = review_data.groupby('week_number', as_index=False)['Rating'].mean()
out2.Rating = out2.Rating.round(1)
out2['Overall Reviews per Week'] = out2['week_number'].map(review_data['week_number'].value_counts())

out3 = out2.drop("week_number", axis='columns')
out3 = out3.pct_change().round(2)


def highlight_max(s): 
    if s.dtype == np.object: 
        is_neg = [False for _ in range(s.shape[0])] 
    else: 
        is_neg = s < 0
    return ['color: red;' if cell else 'color:green' 
            for cell in is_neg] 
  
out_prefinal = out3.style.apply(highlight_max)

out3.rename(columns = {'Rating':'Rating Change in %', 'Overall Reviews per Week':'Overall Reviews per Week Change in %'}, inplace = True)


# Writing Down the Data 
gc = pygsheets.authorize(service_file='/Users/zvezdaetogodoma/Desktop/company-ratings-project-6135c223419a.json')

#open the google spreadsheet (where 'Company_ratings' is the name of my sheet)
sh = gc.open('Company_ratings')

#select the third sheet
wks = sh[1]

#update the first sheet with df, starting at cell B2. 
wks.set_dataframe(out2,(1,1))
wks.set_dataframe(out3,(1,4))

print(out_prefinal)
out_prefinal

/var/folders/kv/xm2scl313w35kcg55mzvr1q80000gn/T/ipykernel_23939/3826845220.py:10: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if s.dtype == np.object:


,Rating Change in %,Overall Reviews per Week Change in %
0,nan,nan
1,0.440000,0.000000
2,-0.210000,0.730000
3,-0.030000,0.050000
4,0.200000,-0.880000


In [691]:
# review_data["Number of Weekly Reviews"] = review_data.groupby(['week_number'])['Rating'].count()
review_data.groupby('week_number')['Rating'].count()

# review_data["Number of Weekly Reviews"] = 
result = review_data.groupby(['week_number','Rating']).count().fillna(0)
result1 = review_data.groupby([pd.Categorical(review_data.week_number), 'Rating']).count().fillna(0)

result2 = result1.drop("date", axis='columns')
result2.rename(columns = {'week_number':'number of reviews'}, inplace = True)
# result2 = result2.drop("Overall Weekly Rating", axis='columns')
# result2 = result2.drop("Number of Weekly Reviews", axis='columns')

df4 = result1.reset_index()
df4.rename(columns = {'level_0':'week_number', 'Rating':'stars', 'date':'quantity of star reviews'}, inplace = True)
# df4 = df4.drop(['Overall Weekly Rating', 'Number of Weekly Reviews'], axis=1)
df4 = df4.iloc[: , :-1]
df4['week_number'] = df4['week_number'].astype('int')

# Writing Down the Data 
gc = pygsheets.authorize(service_file='/Users/zvezdaetogodoma/Desktop/company-ratings-project-6135c223419a.json')

#open the google spreadsheet (where 'Company_ratings' is the name of my sheet)
sh = gc.open('Company_ratings')

#select the third sheet
wks = sh[1]

#update the sheet with df 
wks.set_dataframe(df4,(8,1))

print(df4)

   week_number stars quantity of star reviews
0            1     1                       12
1            1     2                        0
2            1     3                        1
3            1     4                        0
4            1     5                        9
5            2     1                        4
6            2     2                        1
7            2     3                        1
8            2     4                        3
9            2     5                       13
10           3     1                       16
11           3     2                        2
12           3     3                        0
13           3     4                        1
14           3     5                       19
15           4     1                       17
16           4     2                        2
17           4     3                        1
18           4     4                        3
19           4     5                       17
20           5     1              